CONSULTAS NEO4J BASE DE DATOS TEST2

In [ ]:
 # contar todos los tipos de relaciones
MATCH ()-[relationship]->() 
RETURN TYPE(relationship) AS type, COUNT(relationship) AS amount
ORDER BY amount DESC;

In [ ]:
MATCH (n) 
RETURN DISTINCT count(labels(n)), labels(n);

In [ ]:
CALL db.schema.visualization()
#despliega el esquema de la base de datos

In [ ]:
MATCH (e1:Dataset)-[]-(:Sample)[:EG]->(g:Gene)<-[:GDA]-(d:Desease)
match (e2:Experiment{name:'GSM4462415'})
WHERE  NOT (g)<-[:EG]-(e2) and d.name STARTS WITH 'Diabetes'
return g,d;

In [ ]:
MATCH (e1:Experiment{name:'GSM4462413'})-[:EG]->(g:Gene)<-[:GDA]-(d:Desease)
match (e2:Experiment{name:'GSM4462415'})
WHERE  NOT (g)<-[:EG]-(e2) and d.name STARTS WITH 'Diabetes' and (g)-[:Regulates]->()
return g,d,e1,e2 ;

In [ ]:
MATCH (a:Sample)-[r]->(:Gene) RETURN COUNT(r),(a)
#por nodo sample cuantas relaicones tiene cada una

In [ ]:
#si hago un display de Genes, veo toda la red de dorothea, 
#sobre lo anterior consulto para una muestra ¿cual son ls TF(genes que regulan) que estan y los que no estan?,

MATCH (e1:Experiment{name:'GSM4462413'})-[:EXPRESSED]->(g:Gene)-[:REGULATES]->(Gene)
MATCH (e1:Experiment{name:'GSM4462413'}) match (g:Gene)-[:REGULATES]->(Gene) where not (e1)-[:EXPRESSED]->(g:Gene)

#eso me entrega la red contextualizada

In [ ]:
#que factores de regulacion cambian?
MATCH (g:Gene)-[:REGULATES]->(Gene) where (e1:Experiment{name:'GSM4462413'})-[:EXPRESSED]->(g)<-[:EXPRESSED]-(e2:Experiment{name:'GSM4462414'});
MATCH (e1:Experiment{name:'GSM4462413'})-[:EXPRESSED]-(g:Gene)-[:REGULATES]->(Gene) where not (e2)-[:EXPRESSED]->(g:Gene)

In [ ]:
#con la lista de TF que cambian mostra a los genes regulan y por tanto les cambia la regulacion entre condiciones, 
# entre ambos entregar link con comorvilidades
recordar que e1 y e2 son muestras con nombres pero que pueden ser grupos de nodos que cumplen con tags comunes


caso de ejemplo

In [ ]:
MATCH (S1:Sample)<-[HAVE{condition:'Healthy'}]-(b:BioProyect) where b.accession='PRJNA639275',
MATCH (S2:Sample)<-[HAVE{condition:'Infected'}]-(b:BioProyect) where b.accession='PRJNA639275' return S1,S2

In [ ]:
MATCH (s:Sample)-[:HAVE{condition:'Healthy'}]-(bp:BioProyect{accession:'PRJNA639275'})
MATCH (n:TF)-[:EXPRESSED]-(s)
RETURN n,s,bp

In [ ]:
MATCH (n:TF)-[e:EXPRESSED]-(s:Sample{SAMN: 'SAMN15230275'})
match (s2:Sample{SAMN: 'SAMN15230282'})
where not (n)-[:EXPRESSED]-(s2)
RETURN n,s,e,s2,

In [ ]:
MATCH (n:TF)-[e:EXPRESSED]-(s:Sample{SAMN: 'SAMN15230275'})
MATCH (n)<-[d2:GDA]-(d:Desease)
match (s2:Sample{SAMN: 'SAMN15230282'})
where not (n)-[:EXPRESSED]-(s2)
RETURN n,s,e,s2,d

In [ ]:
MATCH (n:TF)-[e:EXPRESSED]-(s:Sample{SAMN: 'SAMN15230275'})
MATCH (n)<-[d2:GDA]-(d:Desease) where d.name contains 'iabetes'
match (s2:Sample{SAMN: 'SAMN15230282'})
where not (n)-[:EXPRESSED]-(s2)
RETURN n,s,e,s2,d,d2

In [ ]:
MATCH (p:Gene) where exist()
<-[:EXPRESSED]-(s:Sample)<-[h:HAS]-()
WHERE h.condition = 'Healthy'
AND NOT exists((friend)-[:WORKS_FOR]->(:Company) where )
RETURN friend.name;

Diferencias en consulta de Neo4j vs SQL

In [ ]:
#(1)
MATCH (s:Sample)-[relationship]->(:Gene) 
RETURN s.SAMN AS Sample, COUNT(relationship) AS amount

In [ ]:
#(1)
SELECT Sample_Description.Sample_code, COUNT(Genes.Gene_ID) as 'Expressed Genes'
from Sample_Description 
inner join Samples_Regulations ON Sample_Description.Sample_ID = Samples_Regulations.Sample_ID
INNER JOIN Genes ON Samples_Regulations.Gene_ID = Genes.Gene_ID
group by Sample_Description.Sample_code

In [ ]:
#(2)
MATCH (s:Sample)-[relationship]->(g:Gene)  where ID(s)=44637 return g.ENSEMBL as Expressed_Genes

In [ ]:
#(2)
SELECT(Genes.Gene_ENGS) as 'Expressed Genes' 
from Sample_Description inner join Samples_Regulations ON Sample_Description.Sample_ID = Samples_Regulations.Sample_ID
INNER JOIN Genes ON Samples_Regulations.Gene_ID = Genes.Gene_ID
where Sample_Description.Sample_ID=1

In [ ]:
#(3) 
match (s:Sample)-[:EXPRESSED]->(g:Gene) where ID(s)=44637 match (g)-[:GDA]-(d:Disease)  return d.name as Diseases_Name

In [ ]:
#(3)
SELECT Sample_Description.Sample_code, Genes.Gene_ENGS, Disease.Disease_Name
from Sample_Description 
INNER JOIN Samples_Regulations ON Sample_Description.Sample_ID = Samples_Regulations.Sample_ID
INNER JOIN Genes ON Samples_Regulations.Gene_ID = Genes.Gene_ID
INNER JOIN GDA ON Genes.Gene_ID=GDA.Gene_ID 
INNER JOIN Disease ON GDA.DIsease_ID=Disease.Disease_ID
where Sample_Description.Sample_ID=1

In [ ]:
#(4)
MATCH (s1:Sample)-[:EXPRESSED]->(g:Gene)<-[:EXPRESSED]-(s2:Sample)  where ID(s1)=44637 and ID(s2)=44638  return g.ENSEMBL as Expressed_Genes

In [ ]:
#(4)
SELECT  Genes.Gene_ENGS, Disease.Disease_Name
from Sample_Description 
INNER JOIN Samples_Regulations ON Sample_Description.Sample_ID = Samples_Regulations.Sample_ID
INNER JOIN Genes ON Samples_Regulations.Gene_ID = Genes.Gene_ID
INNER JOIN GDA ON Genes.Gene_ID=GDA.Gene_ID 
INNER JOIN Disease ON GDA.DIsease_ID=Disease.Disease_ID
where Sample_Description.Sample_ID=1
INTERSECT
SELECT  Genes.Gene_ENGS, Disease.Disease_Name
from Sample_Description 
INNER JOIN Samples_Regulations ON Sample_Description.Sample_ID = Samples_Regulations.Sample_ID
INNER JOIN Genes ON Samples_Regulations.Gene_ID = Genes.Gene_ID
INNER JOIN GDA ON Genes.Gene_ID=GDA.Gene_ID 
INNER JOIN Disease ON GDA.DIsease_ID=Disease.Disease_ID
where Sample_Description.Sample_ID=2

Query in Neo4j fron GRNs

In [ ]:
match (s1:Sample)-[t1:TYPE]-(c1:Condition:Infected)
match (s2:Sample)-[t2:TYPE]-(c2:Condition:Healthy)
match (tf:TF)-[e:EXPRESSED]-(s1) where not (tf)-[:EXPRESSED]-(s2) return s1,s2,t1,t2,c1,c2,tf,e

In [ ]:
match samp1=(s1:Sample)-[t1:TYPE]-(c1:Condition:Healthy)
match samp2=(s2:Sample)-[t2:TYPE]-(c2:Condition:Infected)
match tfs=(tf:TF)-[e:EXPRESSED]-(s1) where not (tf)-[:EXPRESSED]-(s2) 
with tfs 
match(r:TF)-[:GDA]-(d:Disease) where r IN nodes(tfs)  return d

In [ ]:
match samp1=(s1:Sample)-[t1:TYPE]-(c1:Condition:Healthy)
match samp2=(s2:Sample)-[t2:TYPE]-(c2:Condition:Infected)
match tfs=(tf:TF)-[e:EXPRESSED]-(s1) where not (tf)-[:EXPRESSED]-(s2) 
with tfs ,samp1,samp2
match(r1:TF)-[r2:GDA]-(d:Disease) where r1 IN nodes(tfs) and d.name contains 'iabetes' 
return r1,r2, d,nodes(samp1),nodes(samp2),relationships(samp1),relationships(samp2),relationships(tfs)

In [ ]:
match samp1=(s1:Sample)-[t1:TYPE]-(c1:Condition:Healthy)
match samp2=(s2:Sample)-[t2:TYPE]-(c2:Condition:Infected)
match tfs=(tf:TF)-[e:EXPRESSED]-(s1) where not (tf)-[:EXPRESSED]-(s2) 
with tfs ,samp1,samp2
match(r1:TF)-[r2:GDA]-(d:Disease) where r1 IN nodes(tfs) and d.name contains 'tension'
return r1,r2, d,nodes(samp1),nodes(samp2),relationships(samp1),relationships(samp2),relationships(tfs)